In [25]:
from emojify.utils.nn_manager import NNManager
import numpy as np
from emojify.models.lstm_net import LSTMNet
import torch
import emojify.utils as utils
import seaborn as sns
import matplotlib.pyplot as plt
from emojify.dataloaders.transformer_data import load_transformer_data
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
torch.manual_seed(123)
np.random.seed(123)
embedding_dim = 384  # should probably be 32
hidden_dim = 32  # should probably be 32
epochs = 30
batch_size = 32
learning_rate = 2e-3
early_stop_count = 10
train, val, test = load_transformer_data(batch_size)

classes [4, 4, 0, 3, 0, 4, 5, 1, 2, 3, 4, 2, 0, 4, 2, 2, 4, 4, 4, 1, 0, 1, 2, 2, 0, 4, 4, 4, 0, 2, 2, 1, 5, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 4, 4, 2, 3, 2, 0, 2, 4, 0, 1, 2, 4, 4, 5, 2, 2, 2, 3, 1, 1, 5, 0, 0, 4, 3, 2, 4, 4, 2, 4, 4, 4, 2, 2, 2, 0, 4, 0, 0, 0, 2, 2, 2, 2, 4, 1, 3, 0, 4, 0, 3, 4, 2, 2, 4, 0, 3, 2, 2, 2, 4, 2, 2, 2, 2, 4, 2, 2, 3, 4, 4, 2, 2, 4, 2, 2, 1, 1, 1, 4, 3, 2, 2, 3, 1, 5, 2, 2, 2, 2, 0, 1, 2, 0, 3, 0, 4, 2, 4, 0, 2, 5, 4, 0, 0, 4, 2, 1, 2, 2, 1, 4, 5, 5, 2, 0, 1, 0, 4, 0, 4, 1, 4, 2, 5, 1, 2, 0, 2, 0, 2, 1, 1, 4, 2, 2, 4, 5, 2, 1, 2, 4, 2, 2, 2, 1, 2, 5, 2, 0, 4, 4, 2, 0, 4, 2, 2, 5, 2, 1, 2, 1, 4, 1, 3, 4, 1, 3, 4, 4, 0, 2, 3, 0, 4, 5, 0, 2, 4, 0, 2, 2, 2, 2, 2, 4, 2, 3, 2, 4, 0, 5, 2, 0, 2, 3, 4, 4, 2, 4, 4, 2, 4, 2, 2, 2, 1, 4, 0, 4, 0, 3, 2, 3, 4, 2, 2, 2, 4, 2, 4, 0, 3, 4, 3, 4, 4, 4, 4, 4, 4, 1, 2, 4, 4, 2, 3, 4, 0, 4, 3, 2, 2, 2, 4, 2, 4, 2, 2, 2, 4, 4, 4, 3, 1, 4, 1, 2, 2, 1, 2, 0, 0, 3, 4, 5, 2, 0, 4, 4, 0, 3, 5, 4, 3, 2, 0, 2, 4, 4, 2, 2, 1, 4, 0, 2, 3

In [27]:
model = LSTMNet(embedding_dim, hidden_dim, 6)
nn_manager = NNManager(
    batch_size,
    learning_rate,
    early_stop_count,
    epochs,
    model,
    (train, val, test),
)
#nn_manager.train()

LSTMNet(
  (lstm): LSTM(384, 32, num_layers=2, batch_first=True)
  (seq): Sequential(
    (0): ReLU()
    (1): Linear(in_features=32, out_features=128, bias=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=6, bias=True)
  )
)


In [28]:
accuracy = nn_manager.test()
accuracy

100%|██████████| 63/63 [00:00<00:00, 958.50it/s]

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       297
           1       0.11      1.00      0.19       214
           2       0.00      0.00      0.00       663
           3       0.00      0.00      0.00       166
           4       0.00      0.00      0.00       577
           5       0.00      0.00      0.00        83

    accuracy                           0.11      2000
   macro avg       0.02      0.17      0.03      2000
weighted avg       0.01      0.11      0.02      2000




c:\Users\ugioh\code\emojify\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ugioh\code\emojify\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ugioh\code\emojify\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.107

In [33]:
from tqdm import tqdm
from sklearn.metrics import classification_report, accuracy_score
new_model = LSTMNet(embedding_dim, hidden_dim, 6)
new_model.load_state_dict(torch.load("best_checkpoints/best_model.ckpt"))
def test_model(model, data) -> float:
    """Test the model using the test dataset and generates a classification report
    and returns the accuracy score.

    Returns:
        float: The accuracy of the model.
    """
    y_pred = []
    y_true = []
    model.eval()
    with torch.no_grad():
        for X_batch, Y_batch in tqdm(data):
            output = model(X_batch)  # type: ignore
            pred = output.argmax(
                dim=1
            )  # get the index of the max log-probability  # noqa
            y_pred.append(pred)
            y_true.append(Y_batch)
    y_pred = torch.cat(y_pred).cpu().numpy()
    y_true = torch.cat(y_true).cpu().numpy()
    print("Classification Report:")
    print(classification_report(y_true, y_pred))
    return float(accuracy_score(y_true, y_pred))
test_model(new_model, test)


100%|██████████| 63/63 [00:00<00:00, 440.89it/s]

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.85      0.84       297
           1       0.81      0.83      0.82       214
           2       0.91      0.82      0.86       663
           3       0.80      0.69      0.74       166
           4       0.82      0.92      0.87       577
           5       0.72      0.76      0.74        83

    accuracy                           0.84      2000
   macro avg       0.82      0.81      0.81      2000
weighted avg       0.84      0.84      0.84      2000



0.8415

In [30]:
class_to_idx = {'anger': 0, 'fear': 1, 'joy': 2, 'love': 3, 'sadness': 4, 'surprise': 5}
idx_to_class = {v: k for k, v in class_to_idx.items()}
from emojify.transformer_embeddings import get_transformer_embeddings
import emojify.utils as utils

def get_emoji(sentence):
    model.eval()
    with torch.no_grad():
        sentence = get_transformer_embeddings(sentence)
        input_ = utils.to_cuda(torch.tensor(sentence).reshape(1, -1))
        tag_scores = model(input_)
        predicted = tag_scores.argmax(dim=1)
        return idx_to_class[predicted.item()]


ModuleNotFoundError: No module named 'emojify.transformer_embeddings'

In [ ]:
get_emoji("This is a very beautiful day")


'joy'

In [ ]:
get_emoji("I hate you")

'anger'

In [ ]:
get_emoji("What the hell is wrong with you")

'anger'